# GPU 使用 （一）

### 查看NVIDIA显卡状态

一共有四块显卡：`0`, `1`, `2`, `3`，每块显卡的最大显存为12Gb

In [ ]:
! nvidia-smi

### 查看内存使用

一共有64G内存（无交换分区 Swap=0）

In [ ]:
! free -h

### 查看Pytorch是否调用显卡

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

返回Pytorch识别到的显卡数目

In [ ]:
torch.cuda.device_count()

### 数据传输

Before：注意显卡内存占用

In [ ]:
! nvidia-smi

显存(GPU memory)中的数据与内存(CPU memory)中的数据不能直接混用，需要通过`.cuda()`及`.cpu()`来进行数据迁移

In [ ]:
# .cuda() 即可将一个对象分配在显卡中
l = torch.nn.Linear(10, 3).cuda()
X = torch.rand(2, 10).cuda()

In [ ]:
X.device

In [ ]:
X.cpu().device

After：第0块显卡的内存使用增加了

In [ ]:
! nvidia-smi

如果数据不在同一设备上，则会报CUDA RuntimeError

In [ ]:
l = torch.nn.Linear(10, 3) # In * Out
X = torch.rand(2, 10) # N * F
l(X) # N * Out

In [ ]:
l(X.cuda()) # 网络在内存中，数据在显存中

In [ ]:
l.cuda()(X) # 网络在显存中，数据在内存中

In [ ]:
l.cuda(0)(X.cuda(1)) # 网络在GPU0中而数据在GPU1中

一些名词：

* `CUDA` (Compute Unified Device Architecture) 是NVIDIA显卡的并行计算框架

一般来说，为了调用GPU进行深度学习训练，需要安装以下两个工具箱：

* `CUDA Toolkit`
* `CuDNN` (CUDA Deep Neural Network)

判断自己的显卡能否使用GPU进行训练，一般只需要检查显卡支持的CUDA版本以及pytorch支持的CUDA版本是否兼容

当前设备使用的版本为：`CUDA 10.0 cudnn7 Ubuntu16.04` (版本极其重要)

Best practice:

* 数据的传输是很慢的，因此多显卡运算时的一个核心问题在于如何尽可能地减少数据传输
* 除非因为显存限制，否则使用多GPU训练并不十分有效：4块显卡一个网络花费训练2小时 vs 1块显卡训练四个网络花费5小时